# 📌 Passo 1 - Importando bibliotecas necessárias


In [4]:
# Conectar ao banco de dados SQLite
import sqlite3

conn = sqlite3.connect('melhores_compras.db')
cursor = conn.cursor()

print("✅ Conexão com SQLite estabelecida.")


✅ Conexão com SQLite estabelecida.


# 📌 Passo 2 - Remover tabelas se existirem (Drop Tables)

In [5]:
import sqlite3
import time

# Conectar ao banco de dados
conn = sqlite3.connect("melhores_compras.db")
cursor = conn.cursor()

# Lista de comandos DROP TABLE adaptados para SQLite
drop_tables_sql = [
    "DROP TABLE IF EXISTS visualizacao_video;",
    "DROP TABLE IF EXISTS sac;",
    "DROP TABLE IF EXISTS video_produto;",
    "DROP TABLE IF EXISTS cliente;",
    "DROP TABLE IF EXISTS funcionario;",
    "DROP TABLE IF EXISTS produto;"
]

# Executar comandos DROP TABLE com tentativa de desbloqueio
for drop_query in drop_tables_sql:
    retry = True
    while retry:
        try:
            cursor.execute(drop_query)
            conn.commit()
            retry = False
        except sqlite3.OperationalError as e:
            if "database is locked" in str(e).lower():
                print(f"Aguardando liberação do banco para: {drop_query}")
                time.sleep(1)
            else:
                raise e

print("✅ Tabelas existentes removidas com sucesso.")



✅ Tabelas existentes removidas com sucesso.


# 📌 Passo 3 - Confirmar remoção das tabelas


In [6]:
# Verificando se as tabelas foram removidas corretamente
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()

if tabelas:
    print("⚠️ Atenção: Ainda há tabelas existentes:")
    print(tabelas)
else:
    print("✅ Nenhuma tabela existente encontrada. Ambiente limpo.")


⚠️ Atenção: Ainda há tabelas existentes:
[('sqlite_sequence',)]


# 📌 Passo 4 - Fechando conexão SQLite

In [7]:
# Fechar a conexão
conn.close()

print("✅ Conexão com SQLite fechada.")


✅ Conexão com SQLite fechada.


# 📌 Criando a tabela `produto`


In [8]:
import sqlite3

# Reconectando ao SQLite
conn = sqlite3.connect('melhores_compras.db')
cursor = conn.cursor()

# Criando tabela produto
cursor.execute('''
CREATE TABLE IF NOT EXISTS produto (
    id_produto INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    descricao TEXT,
    preco REAL,
    data_cadastro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')

conn.commit()
print("✅ Tabela produto criada com sucesso.")


✅ Tabela produto criada com sucesso.


# 📌 Criando a tabela `cliente`


In [9]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS cliente (
    id_cliente INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    email TEXT UNIQUE,
    telefone TEXT,
    data_cadastro TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
''')

conn.commit()
print("✅ Tabela cliente criada com sucesso.")


✅ Tabela cliente criada com sucesso.


In [10]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())


[('sqlite_sequence',), ('produto',), ('cliente',)]


# 📌 Criando a tabela `video_produto`


In [11]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS video_produto (
    id_video INTEGER PRIMARY KEY AUTOINCREMENT,
    id_produto INTEGER NOT NULL,
    url_video TEXT NOT NULL,
    descricao TEXT,
    status TEXT CHECK(status IN ('A','I')) DEFAULT 'A',
    data_cadastro TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto) ON DELETE CASCADE
)
''')

conn.commit()
print("✅ Tabela video_produto criada com sucesso.")


✅ Tabela video_produto criada com sucesso.


# 📌 Criando a tabela `funcionario`


In [12]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS funcionario (
    id_funcionario INTEGER PRIMARY KEY AUTOINCREMENT,
    nome TEXT NOT NULL,
    cpf TEXT NOT NULL UNIQUE,
    data_nascimento DATE NOT NULL,
    telefone TEXT NOT NULL,
    email TEXT NOT NULL,
    cargo TEXT NOT NULL,
    departamento TEXT NOT NULL
)
''')

conn.commit()
print("✅ Tabela funcionario criada com sucesso.")


✅ Tabela funcionario criada com sucesso.


# 📌 Criando a tabela `visualizacao_video`


In [13]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS visualizacao_video (
    id_visualizacao INTEGER PRIMARY KEY AUTOINCREMENT,
    id_video INTEGER NOT NULL,
    id_cliente INTEGER,
    data_hora_visualizacao TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    FOREIGN KEY (id_video) REFERENCES video_produto(id_video) ON DELETE CASCADE,
    FOREIGN KEY (id_cliente) REFERENCES cliente(id_cliente) ON DELETE SET NULL
)
''')

conn.commit()
print("✅ Tabela visualizacao_video criada com sucesso.")


✅ Tabela visualizacao_video criada com sucesso.


# 📌 Criando a tabela `sac` (Serviço de Atendimento ao Cliente)


In [14]:
cursor.execute('''
CREATE TABLE IF NOT EXISTS sac (
    id_sac INTEGER PRIMARY KEY AUTOINCREMENT,
    id_cliente INTEGER NOT NULL,
    id_produto INTEGER NOT NULL,
    id_funcionario INTEGER NOT NULL,
    data_hora_abertura TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_hora_atendimento TIMESTAMP,
    tempo_total_atendimento INTEGER,
    status_chamado TEXT NOT NULL CHECK (status_chamado IN ('A','E','C','F','X')),
    tipo_chamado TEXT NOT NULL CHECK (tipo_chamado IN ('1','2')),
    indice_satisfacao INTEGER CHECK (indice_satisfacao BETWEEN 1 AND 100),
    descricao_chamado TEXT NOT NULL,
    FOREIGN KEY (id_cliente) REFERENCES cliente(id_cliente) ON DELETE CASCADE,
    FOREIGN KEY (id_produto) REFERENCES produto(id_produto) ON DELETE CASCADE,
    FOREIGN KEY (id_funcionario) REFERENCES funcionario(id_funcionario) ON DELETE CASCADE
)
''')

conn.commit()
print("✅ Tabela sac criada com sucesso.")


✅ Tabela sac criada com sucesso.


# 📌 Conferindo tabelas criadas no banco SQLite


In [15]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas_criadas = cursor.fetchall()

print("✅ Tabelas criadas com sucesso no banco SQLite:")
for tabela in tabelas_criadas:
    print(tabela[0])


✅ Tabelas criadas com sucesso no banco SQLite:
sqlite_sequence
produto
cliente
video_produto
funcionario
visualizacao_video
sac


# 📌 Fechando conexão com o SQLite


In [16]:
# Fechar a conexão SQLite
conn.close()
print("✅ Conexão com SQLite fechada com sucesso.")


✅ Conexão com SQLite fechada com sucesso.


## 📌 Desafio 1: Inserindo produtos na tabela `produto`

In [17]:
import sqlite3

# Reconectando ao banco SQLite
conn = sqlite3.connect('melhores_compras.db')
cursor = conn.cursor()

# Produtos para inserção (exemplos fictícios)
produtos = [
    ('Smartphone XYZ', 'Smartphone XYZ com 128GB, câmera 50MP', 1799.99),
    ('Notebook ABC', 'Notebook ABC com processador i7, 16GB RAM', 4599.50),
    ('Smart TV LED 50"', 'Smart TV com resolução 4K Ultra HD', 2799.00),
]

# Inserindo produtos
cursor.executemany('''
INSERT INTO produto (nome, descricao, preco)
VALUES (?, ?, ?)
''', produtos)

conn.commit()

print("✅ Produtos inseridos com sucesso.")


✅ Produtos inseridos com sucesso.


## 📌 Desafio 2: Inserindo clientes na tabela `cliente`

In [18]:
# Clientes para inserção
clientes = [
    ('Alice Silva', 'alice.silva@example.com', '(11)98765-4321'),
    ('Bruno Souza', 'bruno.souza@example.com', '(21)91234-5678'),
    ('Carla Mendes', 'carla.mendes@example.com', '(31)97654-3210'),
]

# Inserindo clientes
cursor.executemany('''
INSERT INTO cliente (nome, email, telefone)
VALUES (?, ?, ?)
''', clientes)

conn.commit()

print("✅ Clientes inseridos com sucesso.")


✅ Clientes inseridos com sucesso.


## 📌 Desafio 3: Inserindo funcionários na tabela `funcionario`

funcionários que serão responsáveis pelo atendimento ao cliente (SAC).


In [19]:
# Funcionários para inserção
funcionarios = [
    ('João Pereira', '12345678901', '1985-04-23', '(11)91234-5678', 'joao.pereira@example.com', 'Analista de SAC', 'Atendimento'),
    ('Maria Gomes', '10987654321', '1990-09-10', '(21)92345-6789', 'maria.gomes@example.com', 'Supervisora de SAC', 'Atendimento'),
]

# Inserindo funcionários
cursor.executemany('''
INSERT INTO funcionario (nome, cpf, data_nascimento, telefone, email, cargo, departamento)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', funcionarios)

conn.commit()

print("✅ Funcionários inseridos com sucesso.")


✅ Funcionários inseridos com sucesso.


## 📌 Conferência dos dados inseridos

In [20]:
import pandas as pd

# Conferir tabela produto
print("📌 Produtos cadastrados:")
df_produto = pd.read_sql_query("SELECT * FROM produto", conn)
display(df_produto)

# Conferir tabela cliente
print("📌 Clientes cadastrados:")
df_cliente = pd.read_sql_query("SELECT * FROM cliente", conn)
display(df_cliente)

# Conferir tabela funcionario
print("📌 Funcionários cadastrados:")
df_funcionario = pd.read_sql_query("SELECT * FROM funcionario", conn)
display(df_funcionario)


📌 Produtos cadastrados:


,id_produto,nome,descricao,preco,data_cadastro
0,1,Smartphone XYZ,"Smartphone XYZ com 128GB, câmera 50MP",1799.99,2025-04-21 15:40:25
1,2,Notebook ABC,"Notebook ABC com processador i7, 16GB RAM",4599.50,2025-04-21 15:40:25
2,3,"Smart TV LED 50""",Smart TV com resolução 4K Ultra HD,2799.00,2025-04-21 15:40:25


📌 Clientes cadastrados:


,id_cliente,nome,email,telefone,data_cadastro
0,1,Alice Silva,alice.silva@example.com,(11)98765-4321,2025-04-21 15:40:25
1,2,Bruno Souza,bruno.souza@example.com,(21)91234-5678,2025-04-21 15:40:25
2,3,Carla Mendes,carla.mendes@example.com,(31)97654-3210,2025-04-21 15:40:25


📌 Funcionários cadastrados:


,id_funcionario,nome,cpf,data_nascimento,telefone,email,cargo,departamento
0,1,João Pereira,12345678901,1985-04-23,(11)91234-5678,joao.pereira@example.com,Analista de SAC,Atendimento
1,2,Maria Gomes,10987654321,1990-09-10,(21)92345-6789,maria.gomes@example.com,Supervisora de SAC,Atendimento


## 📌 Fechando conexão após inserções iniciais

Encerrando a conexão segura com o SQLite após confirmação das inserções.


In [21]:
conn.close()
print("✅ Conexão fechada após inserções iniciais.")


✅ Conexão fechada após inserções iniciais.


## 📌 Inserindo vídeos relacionados aos produtos cadastrados


In [22]:
import sqlite3

# Reconectando ao SQLite
conn = sqlite3.connect('melhores_compras.db')
cursor = conn.cursor()

# IDs dos produtos inseridos anteriormente (confirmar IDs antes de rodar!)
# Exemplo genérico assumindo que IDs são 1, 2 e 3 respectivamente
videos_produtos = [
    (1, 'https://youtube.com/video-smartphone-xyz', 'Review Smartphone XYZ', 'A'),
    (2, 'https://youtube.com/video-notebook-abc', 'Unboxing Notebook ABC', 'A'),
    (3, 'https://youtube.com/video-smart-tv-led', 'Análise Smart TV 4K', 'I'),  # Inativo como exemplo
]

cursor.executemany('''
INSERT INTO video_produto (id_produto, url_video, descricao, status)
VALUES (?, ?, ?, ?)
''', videos_produtos)

conn.commit()
print("✅ Vídeos relacionados aos produtos inseridos com sucesso.")


✅ Vídeos relacionados aos produtos inseridos com sucesso.


## 📌 Registrando visualizações de vídeos

Simulando visualizações realizadas pelos clientes cadastrados e também por usuários anônimos.


In [23]:
from datetime import datetime

# Simulando visualizações (id_video, id_cliente, data_hora_visualizacao)
# Vamos assumir que clientes têm id 1, 2 e 3
visualizacoes = [
    (1, 1, datetime.now()),
    (2, 2, datetime.now()),
    (1, None, datetime.now()),  # Anônimo assistindo vídeo
]

cursor.executemany('''
INSERT INTO visualizacao_video (id_video, id_cliente, data_hora_visualizacao)
VALUES (?, ?, ?)
''', visualizacoes)

conn.commit()
print("✅ Visualizações de vídeos registradas com sucesso.")


✅ Visualizações de vídeos registradas com sucesso.


## 📌 Registrando chamados de SAC

Simulando abertura de chamados de atendimento ao cliente (SAC).


In [24]:
# Chamados para inserção (exemplos fictícios)
# Assumindo id_cliente 1, id_produto 1, id_funcionario 1
chamados_sac = [
    (1, 1, 1, datetime.now(), None, None, 'A', '2', None, 'Problema na câmera do Smartphone XYZ'),
    (2, 2, 2, datetime.now(), datetime.now(), 30, 'F', '1', 90, 'Sugestão para melhorar o Notebook ABC'),
]

cursor.executemany('''
INSERT INTO sac (
    id_cliente, id_produto, id_funcionario, data_hora_abertura, data_hora_atendimento,
    tempo_total_atendimento, status_chamado, tipo_chamado, indice_satisfacao, descricao_chamado
) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', chamados_sac)

conn.commit()
print("✅ Chamados do SAC registrados com sucesso.")


✅ Chamados do SAC registrados com sucesso.


## 📌 Conferindo novos dados inseridos

Agora, vamos verificar se vídeos, visualizações e chamados do SAC foram inseridos corretamente.


In [25]:
import pandas as pd

# Conferir vídeos cadastrados
print("📌 Vídeos cadastrados:")
df_videos = pd.read_sql_query("SELECT * FROM video_produto", conn)
display(df_videos)

# Conferir visualizações cadastradas
print("📌 Visualizações registradas:")
df_visualizacoes = pd.read_sql_query("SELECT * FROM visualizacao_video", conn)
display(df_visualizacoes)

# Conferir chamados SAC cadastrados
print("📌 Chamados SAC registrados:")
df_sac = pd.read_sql_query("SELECT * FROM sac", conn)
display(df_sac)


📌 Vídeos cadastrados:


,id_video,id_produto,url_video,descricao,status,data_cadastro
0,1,1,https://youtube.com/video-smartphone-xyz,Review Smartphone XYZ,A,2025-04-21 15:40:30
1,2,2,https://youtube.com/video-notebook-abc,Unboxing Notebook ABC,A,2025-04-21 15:40:30
2,3,3,https://youtube.com/video-smart-tv-led,Análise Smart TV 4K,I,2025-04-21 15:40:30


📌 Visualizações registradas:


,id_visualizacao,id_video,id_cliente,data_hora_visualizacao
0,1,1,1.0,2025-04-21 12:40:30.397822
1,2,2,2.0,2025-04-21 12:40:30.397822
2,3,1,NaN,2025-04-21 12:40:30.397822


📌 Chamados SAC registrados:


,id_sac,id_cliente,id_produto,id_funcionario,data_hora_abertura,data_hora_atendimento,tempo_total_atendimento,status_chamado,tipo_chamado,indice_satisfacao,descricao_chamado
0,1,1,1,1,2025-04-21 12:40:30.430184,None,NaN,A,2,NaN,Problema na câmera do Smartphone XYZ
1,2,2,2,2,2025-04-21 12:40:30.430184,2025-04-21 12:40:30.430184,30.0,F,1,90.0,Sugestão para melhorar o Notebook ABC


## 📌 Fechando conexão após etapa de vídeos, visualizações e SAC


In [26]:
conn.close()
print("✅ Conexão fechada após completar vídeos, visualizações e SAC.")


✅ Conexão fechada após completar vídeos, visualizações e SAC.


## 📌 Consulta 1: Produtos com vídeos ativos
Listar todos os produtos que possuem pelo menos um vídeo com status 'A' (ativo).


In [27]:
import sqlite3
import pandas as pd

# Conexão
conn = sqlite3.connect('melhores_compras.db')

query = '''
SELECT p.id_produto, p.nome AS nome_produto, v.id_video, v.url_video, v.status
FROM produto p
JOIN video_produto v ON p.id_produto = v.id_produto
WHERE v.status = 'A'
'''

df_videos_ativos = pd.read_sql_query(query, conn)
df_videos_ativos


,id_produto,nome_produto,id_video,url_video,status
0,1,Smartphone XYZ,1,https://youtube.com/video-smartphone-xyz,A
1,2,Notebook ABC,2,https://youtube.com/video-notebook-abc,A


## 📌 Consulta 2: Contagem de visualizações por vídeo
Exibir quantas vezes cada vídeo foi visualizado, incluindo vídeos sem visualização.


In [28]:
query = '''
SELECT v.id_video, v.descricao, COUNT(vv.id_visualizacao) AS total_visualizacoes
FROM video_produto v
LEFT JOIN visualizacao_video vv ON v.id_video = vv.id_video
GROUP BY v.id_video, v.descricao
'''

df_visualizacoes = pd.read_sql_query(query, conn)
df_visualizacoes


,id_video,descricao,total_visualizacoes
0,1,Review Smartphone XYZ,2
1,2,Unboxing Notebook ABC,1
2,3,Análise Smart TV 4K,0


## 📌 Consulta 3: Chamados de SAC por cliente
Exibir quantos chamados cada cliente abriu no SAC.


In [29]:

query = '''
SELECT c.nome AS cliente, COUNT(s.id_sac) AS total_chamados
FROM cliente c
JOIN sac s ON c.id_cliente = s.id_cliente
GROUP BY c.nome
'''

df_chamados_por_cliente = pd.read_sql_query(query, conn)
df_chamados_por_cliente


,cliente,total_chamados
0,Alice Silva,1
1,Bruno Souza,1


## 📌 Consulta 4: Distribuição de chamados por tipo e status
Mostrar uma matriz cruzada com tipo de chamado (sugestão ou reclamação) versus status (aberto, fechado, etc).


In [30]:
query = '''
SELECT
  tipo_chamado,
  status_chamado,
  COUNT(*) AS total
FROM sac
GROUP BY tipo_chamado, status_chamado
'''

df_tipo_status = pd.read_sql_query(query, conn)
df_tipo_status


,tipo_chamado,status_chamado,total
0,1,F,1
1,2,A,1


## 📌 Consulta 5: Satisfação média por funcionário
Obter a média do índice de satisfação dos chamados atendidos por cada funcionário.


In [31]:
query = '''
SELECT f.nome AS funcionario, AVG(s.indice_satisfacao) AS media_satisfacao
FROM sac s
JOIN funcionario f ON s.id_funcionario = f.id_funcionario
WHERE s.indice_satisfacao IS NOT NULL
GROUP BY f.nome
'''

df_satisfacao_funcionario = pd.read_sql_query(query, conn)
df_satisfacao_funcionario


,funcionario,media_satisfacao
0,Maria Gomes,90.0


## 📌 Consulta 6: Clientes que visualizaram vídeos mas não abriram SAC
Listar os clientes que visualizaram vídeos e nunca abriram chamados.


In [32]:
query = '''
SELECT DISTINCT c.id_cliente, c.nome
FROM visualizacao_video vv
JOIN cliente c ON vv.id_cliente = c.id_cliente
WHERE c.id_cliente NOT IN (
    SELECT id_cliente FROM sac
)
'''

df_clientes_sem_sac = pd.read_sql_query(query, conn)
df_clientes_sem_sac


,id_cliente,nome


In [33]:
query = '''
SELECT vv.id_visualizacao, vv.id_video, vv.id_cliente, c.nome AS nome_cliente
FROM visualizacao_video vv
LEFT JOIN cliente c ON vv.id_cliente = c.id_cliente
'''

df_visualizacoes = pd.read_sql_query(query, conn)
df_visualizacoes


,id_visualizacao,id_video,id_cliente,nome_cliente
0,1,1,1.0,Alice Silva
1,2,2,2.0,Bruno Souza
2,3,1,NaN,None


query = '''
SELECT DISTINCT c.id_cliente, c.nome
FROM visualizacao_video vv
JOIN cliente c ON vv.id_cliente = c.id_cliente
WHERE c.id_cliente NOT IN (
    SELECT id_cliente FROM sac
)
'''

df_clientes_sem_sac = pd.read_sql_query(query, conn)
df_clientes_sem_sac


In [34]:
# Reconectar ao banco de dados SQLite
conn = sqlite3.connect('melhores_compras.db')
cursor = conn.cursor()

# Inserir cliente
cursor.execute('''
INSERT INTO cliente (nome, email, telefone)
VALUES (?, ?, ?)
''', ('Carla Mendes', 'carla.nova@example.com', '(31)98888-7766'))

conn.commit()

print("✅ Cliente inserido com sucesso.")


✅ Cliente inserido com sucesso.


In [35]:
df_clientes = pd.read_sql_query("SELECT id_cliente, nome, email FROM cliente", conn)
df_clientes


,id_cliente,nome,email
0,1,Alice Silva,alice.silva@example.com
1,2,Bruno Souza,bruno.souza@example.com
2,3,Carla Mendes,carla.mendes@example.com
3,4,Carla Mendes,carla.nova@example.com


In [36]:
cursor.execute("DELETE FROM cliente WHERE email = 'carla.mendes@example.com'")
conn.commit()


 Inserção segura de cliente (evita duplicação)

In [37]:
def inserir_cliente_unico(nome, email, telefone, conn):
    cursor = conn.cursor()
    
    # Verifica se o cliente já existe
    cursor.execute('SELECT id_cliente FROM cliente WHERE email = ?', (email,))
    resultado = cursor.fetchone()
    
    if resultado:
        print(f"⚠️ Cliente já existe: {email}")
        id_cliente = resultado[0]
    else:
        cursor.execute('''
            INSERT INTO cliente (nome, email, telefone)
            VALUES (?, ?, ?)
        ''', (nome, email, telefone))
        conn.commit()
        id_cliente = cursor.lastrowid
        print(f"✅ Novo cliente inserido: {nome} (ID: {id_cliente})")
    
    return id_cliente


In [38]:
# Conectar ao banco se ainda não estiver conectado
conn = sqlite3.connect('melhores_compras.db')

# Inserir Carla (ou recuperar ID se já existir)
id_carla = inserir_cliente_unico(
    nome='Carla Mendes',
    email='carla.mendes@example.com',
    telefone='(31)98888-7766',
    conn=conn
)

print("🆔 ID da Carla:", id_carla)


✅ Novo cliente inserido: Carla Mendes (ID: 5)
🆔 ID da Carla: 5


## 📌 Visualizando chamados em aberto


In [39]:
import pandas as pd

query = '''
SELECT id_sac, id_cliente, id_produto, id_funcionario, data_hora_abertura, status_chamado
FROM sac
WHERE status_chamado = 'A'
'''

df_chamados_abertos = pd.read_sql_query(query, conn)
df_chamados_abertos


,id_sac,id_cliente,id_produto,id_funcionario,data_hora_abertura,status_chamado
0,1,1,1,1,2025-04-21 12:40:30.430184,A


## 📌 Iniciando atendimento ao chamado (status = 'E')


In [40]:
from datetime import datetime

id_chamado = 1  # substitua pelo ID real de um chamado em aberto

data_atendimento = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

cursor.execute('''
UPDATE sac
SET status_chamado = 'E',
    data_hora_atendimento = ?
WHERE id_sac = ?
''', (data_atendimento, id_chamado))

conn.commit()
print(f"✅ Chamado {id_chamado} em atendimento iniciado às {data_atendimento}.")


✅ Chamado 1 em atendimento iniciado às 2025-04-21 12:40:30.


## 📌 Finalizando atendimento do chamado


In [41]:
# Simulando tempo total em minutos
tempo_total = 45
indice_satisfacao = 85  # entre 1 e 100
status_final = 'F'  # ou 'X' para insatisfação

cursor.execute('''
UPDATE sac
SET status_chamado = ?, 
    tempo_total_atendimento = ?, 
    indice_satisfacao = ?
WHERE id_sac = ?
''', (status_final, tempo_total, indice_satisfacao, id_chamado))

conn.commit()
print(f"✅ Chamado {id_chamado} finalizado com sucesso (status: {status_final}).")


✅ Chamado 1 finalizado com sucesso (status: F).


## 📌 Conferindo status atualizado do chamado


In [42]:
df_chamado_finalizado = pd.read_sql_query(
    f"SELECT * FROM sac WHERE id_sac = {id_chamado}", conn
)
df_chamado_finalizado


,id_sac,id_cliente,id_produto,id_funcionario,data_hora_abertura,data_hora_atendimento,tempo_total_atendimento,status_chamado,tipo_chamado,indice_satisfacao,descricao_chamado
0,1,1,1,1,2025-04-21 12:40:30.430184,None,None,A,2,None,Problema na câmera do Smartphone XYZ


## 📌 Relatório de desempenho dos funcionários no SAC


In [43]:
query = '''
SELECT f.nome AS funcionario,
       COUNT(s.id_sac) AS total_chamados,
       AVG(s.tempo_total_atendimento) AS tempo_medio,
       AVG(s.indice_satisfacao) AS satisfacao_media
FROM sac s
JOIN funcionario f ON s.id_funcionario = f.id_funcionario
WHERE s.status_chamado IN ('F','X')
GROUP BY f.nome
'''

df_relatorio = pd.read_sql_query(query, conn)
df_relatorio


,funcionario,total_chamados,tempo_medio,satisfacao_media
0,Maria Gomes,1,30.0,90.0


## 📊 Gráfico: Distribuição de chamados por status


In [44]:
import plotly.express as px

query = '''
SELECT status_chamado, COUNT(*) AS total
FROM sac
GROUP BY status_chamado
'''

df_status = pd.read_sql_query(query, conn)

fig = px.pie(df_status, names='status_chamado', values='total',
             title='Distribuição dos chamados por status',
             hole=0.4)
fig.show()


### 🧾 Bloco 2 – Tabelas de Endereços e Clientes

In [45]:
# Tabelas: cli_fisica, cli_juridica, end_cli
cursor.execute("""
CREATE TABLE IF NOT EXISTS cli_fisica (
    id_cliente INTEGER PRIMARY KEY,
    cpf TEXT NOT NULL,
    data_nascimento DATE,
    FOREIGN KEY (id_cliente) REFERENCES cliente(id_cliente)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS cli_juridica (
    id_cliente INTEGER PRIMARY KEY,
    cnpj TEXT NOT NULL,
    razao_social TEXT,
    FOREIGN KEY (id_cliente) REFERENCES cliente(id_cliente)
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS end_cli (
    id_end_cli INTEGER PRIMARY KEY AUTOINCREMENT,
    id_cliente INTEGER NOT NULL,
    id_logradouro INTEGER NOT NULL,
    numero TEXT,
    complemento TEXT,
    FOREIGN KEY (id_cliente) REFERENCES cliente(id_cliente),
    FOREIGN KEY (id_logradouro) REFERENCES logradouro(id_logradouro)
);
""")

conn.commit()
print("✅ Tabelas de endereço e clientes criadas com sucesso.")

✅ Tabelas de endereço e clientes criadas com sucesso.


In [46]:

# 📋 Verificando tabelas criadas (evidência FIAP) no VS Code para nbconvert

import sqlite3, os
db_path = os.path.join("..", "data", "melhores_compras.db")
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tabelas = cursor.fetchall()

print("Tabelas criadas neste notebook:")
for t in tabelas:
    print(" -", t[0])

conn.close()


Tabelas criadas neste notebook:
 - sqlite_sequence
 - produto
 - video_produto
 - cliente
 - funcionario
 - visualizacao_video
 - sac
 - auditoria_sac
 - depto
 - end_func
 - categoria
 - produto_video
 - imagem_produto
 - satisfacao
